In [1]:
import os
import tarfile
import boto3
import json
from urllib.parse import urlparse

import tensorflow as tf
import sagemaker
from sagemaker.tensorflow import TensorFlowModel

# =========================
# Config
# =========================
H5_TAR_S3_URI = "s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-18-11-02-51-500/output/model.tar.gz"
ENDPOINT_NAME = "BMI-predcitor-v8"
INSTANCE_TYPE = "ml.g4dn.xlarge"
FRAMEWORK_VERSION = "2.11.0"
SAVEDMODEL_TAR_S3_URI = "s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-18-11-02-51-500/output/savedmodel/model.tar.gz"

# Scaler URIs
ROBUST_SCALER_URI = "s3://ai-bmi-predictor/scalers/scaler_robust_features.pkl"
STANDARD_SCALER_URI = "s3://ai-bmi-predictor/scalers/scaler_standard_features.pkl"
TARGET_SCALER_URI = "s3://ai-bmi-predictor/scalers/scaler_targets.pkl"

# =========================
# Role inference
# =========================
try:
    ROLE  # type: ignore
except NameError:
    try:
        from sagemaker import get_execution_role
        ROLE = get_execution_role()
    except Exception:
        ROLE = None

if not ROLE:
    raise ValueError("ROLE is None. Set ROLE to your SageMaker execution role ARN.")


2025-12-18 17:25:28.717586: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-18 17:25:28.733553: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-18 17:25:28.758611: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-18 17:25:28.758651: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-18 17:25:28.774188: I tensorflow/core/platform/cpu_feature_gua

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# =========================
# Helpers
# =========================
def parse_s3_uri(uri: str):
    # Parse S3 URI into bucket and key components
    p = urlparse(uri)
    if p.scheme != "s3" or not p.netloc or not p.path:
        raise ValueError(f"Invalid S3 URI: {uri}")
    return p.netloc, p.path.lstrip("/")

def safe_extract(tar: tarfile.TarFile, path: str):
    # Safely extract tar file preventing path traversal attacks
    abs_path = os.path.abspath(path)
    for member in tar.getmembers():
        member_path = os.path.abspath(os.path.join(path, member.name))
        if not member_path.startswith(abs_path + os.sep) and member_path != abs_path:
            raise Exception(f"Blocked path traversal attempt: {member.name}")
    tar.extractall(path)

def upload_file_to_s3(local_path: str, s3_uri: str):
    # Upload local file to S3
    bucket, key = parse_s3_uri(s3_uri)
    boto3.client("s3").upload_file(local_path, bucket, key)
    return f"s3://{bucket}/{key}"

def download_from_s3(s3_uri: str, local_path: str):
    # Download file from S3 to local path
    bucket, key = parse_s3_uri(s3_uri)
    boto3.client("s3").download_file(bucket, key, local_path)
    print(f"✅ Downloaded: {s3_uri} to {local_path}")

In [3]:
# =========================
# 1) Download & extract .h5 from tar.gz
# =========================
workdir = "bmi_model_work"
os.makedirs(workdir, exist_ok=True)

local_in_tar = os.path.join(workdir, "model.tar.gz")
extract_dir = os.path.join(workdir, "extracted")
os.makedirs(extract_dir, exist_ok=True)

# Download model tar.gz from S3
in_bucket, in_key = parse_s3_uri(H5_TAR_S3_URI)
boto3.client("s3").download_file(in_bucket, in_key, local_in_tar)
print(f"✅ Downloaded: {H5_TAR_S3_URI}")

# Extract tar.gz file
with tarfile.open(local_in_tar, "r:gz") as tar:
    safe_extract(tar, extract_dir)

# Find the .h5 file in extracted directory
h5_path = None
for root, _, files in os.walk(extract_dir):
    for f in files:
        if f.lower().endswith((".h5", ".hdf5")):
            h5_path = os.path.join(root, f)
            break
    if h5_path:
        break

if not h5_path:
    raise ValueError("❌ No .h5 found inside the downloaded archive.")

print(f"✅ Found H5: {h5_path}")

✅ Downloaded: s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-18-11-02-51-500/output/model.tar.gz
✅ Found H5: bmi_model_work/extracted/eff_ann_version8.h5


In [4]:
# =========================
# 2) Download scalers from S3
# =========================
scaler_dir = os.path.join(workdir, "scalers")
os.makedirs(scaler_dir, exist_ok=True)

# Download robust scaler for CNN features
robust_scaler_path = os.path.join(scaler_dir, "scaler_robust_features.pkl")
download_from_s3(ROBUST_SCALER_URI, robust_scaler_path)

# Download standard scaler for height
standard_scaler_path = os.path.join(scaler_dir, "scaler_standard_features.pkl")
download_from_s3(STANDARD_SCALER_URI, standard_scaler_path)

# Download target scaler for inverse transformation
target_scaler_path = os.path.join(scaler_dir, "scaler_targets.pkl")
download_from_s3(TARGET_SCALER_URI, target_scaler_path)


✅ Downloaded: s3://ai-bmi-predictor/scalers/scaler_robust_features.pkl to bmi_model_work/scalers/scaler_robust_features.pkl
✅ Downloaded: s3://ai-bmi-predictor/scalers/scaler_standard_features.pkl to bmi_model_work/scalers/scaler_standard_features.pkl
✅ Downloaded: s3://ai-bmi-predictor/scalers/scaler_targets.pkl to bmi_model_work/scalers/scaler_targets.pkl


In [5]:
# =========================# =========================
# 3) Create inference.py script
# =========================
code_dir = os.path.join(workdir, "code")
os.makedirs(code_dir, exist_ok=True)

inference_script = '''
import os
import json
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import EfficientNetB7, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
from PIL import Image
import io
import base64

# Define target columns in the expected output order
TARGET_COLS = [
    'ankle', 'arm-length', 'bicep', 'calf', 'chest', 'forearm', 'hip',
    'leg-length', 'shoulder-breadth', 'shoulder-to-crotch', 'thigh',
    'waist', 'wrist', 'weight_kg'
]

# Global variables to store loaded models and scalers
feature_extractor = None
bmi_model = None
robust_scaler = None
standard_scaler = None
target_scaler = None

def model_fn(model_dir):
    """Load the model and scalers when the container starts"""
    global feature_extractor, bmi_model, robust_scaler, standard_scaler, target_scaler
    
    # Load EfficientNetB7 for feature extraction
    base_model = EfficientNetB7(weights='imagenet')
    # Create feature extractor using second-to-last layer
    feature_extractor = Model(
        inputs=base_model.inputs,
        outputs=base_model.layers[-2].output
    )
    print("✅ Loaded EfficientNetB7 feature extractor")
    
    # Load the BMI prediction model
    model_path = os.path.join(model_dir, '1')  # TF Serving convention
    bmi_model = tf.keras.models.load_model(model_path)
    print(f"✅ Loaded BMI model from {model_path}")
    
    # Load scalers from the model directory
    scaler_dir = model_dir
    
    # Load robust scaler for CNN extracted features
    with open(os.path.join(scaler_dir, 'scaler_robust_features.pkl'), 'rb') as f:
        robust_scaler = pickle.load(f)
    print("✅ Loaded robust scaler")
    
    # Load standard scaler for height feature
    with open(os.path.join(scaler_dir, 'scaler_standard_features.pkl'), 'rb') as f:
        standard_scaler = pickle.load(f)
    print("✅ Loaded standard scaler")
    
    # Load target scaler for inverse transformation
    with open(os.path.join(scaler_dir, 'scaler_targets.pkl'), 'rb') as f:
        target_scaler = pickle.load(f)
    print("✅ Loaded target scaler")
    
    return bmi_model

def extract_features_from_image(image_data, target_size=(600, 600)):
    """Extract features from a single image using EfficientNetB7"""
    # Convert bytes to PIL Image
    img = Image.open(io.BytesIO(image_data))
    # Resize to target size
    img = img.resize(target_size)
    # Convert to array
    img_array = img_to_array(img)
    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)
    # Preprocess for EfficientNet
    img_array = preprocess_input(img_array)
    # Extract features
    features = feature_extractor.predict(img_array, verbose=0)
    # Flatten to 1D array
    return features.flatten()

def input_fn(request_body, request_content_type):
    """Parse input data"""
    if request_content_type == 'application/json':
        # Parse JSON input
        input_data = json.loads(request_body)
        
        # Decode base64 images
        front_img_data = base64.b64decode(input_data['front_image'])
        side_img_data = base64.b64decode(input_data['side_image'])
        
        # Get gender and height
        gender = input_data['gender']  # expects 'male' or 'female' string
        height_cm = float(input_data['height_cm'])
        
        return {
            'front_image': front_img_data,
            'side_image': side_img_data,
            'gender': gender,
            'height_cm': height_cm
        }
    else:
        raise ValueError(f"Unsupported content type: {request_content_type}")

def predict_fn(input_data, model):
    """Perform prediction with preprocessing and postprocessing"""
    # Extract features from front mask image
    front_features = extract_features_from_image(input_data['front_image'])
    print(f"✅ Extracted front features: {front_features.shape}")
    
    # Extract features from side mask image
    side_features = extract_features_from_image(input_data['side_image'])
    print(f"✅ Extracted side features: {side_features.shape}")
    
    # Encode gender: male=1, female=0 (adjust based on your training)
    gender = input_data['gender'].lower()
    gender_encoded = 1 if gender == 'male' else 0
    
    # Get height
    height_cm = input_data['height_cm']
    
    # Combine all features
    # Order: front_features + side_features + gender + height_cm
    combined_features = np.concatenate([
        front_features,
        side_features,
        [gender_encoded],
        [height_cm]
    ])
    
    # Determine feature indices for scaling
    num_front = len(front_features)
    num_side = len(side_features)
    total_cnn_features = num_front + num_side
    
    # Extract CNN features (front + side)
    cnn_features = combined_features[:total_cnn_features]
    # Gender (no scaling needed)
    gender_feature = combined_features[total_cnn_features:total_cnn_features+1]
    # Height
    height_feature = combined_features[total_cnn_features+1:total_cnn_features+2]
    
    # Apply robust scaler to CNN features
    cnn_features_scaled = robust_scaler.transform(cnn_features.reshape(1, -1)).flatten()
    
    # Apply standard scaler to height
    height_scaled = standard_scaler.transform(height_feature.reshape(1, -1)).flatten()
    
    # Combine scaled features
    final_features = np.concatenate([
        cnn_features_scaled,
        gender_feature,
        height_scaled
    ])
    
    # Reshape for model input
    final_features = final_features.reshape(1, -1)
    print(f"✅ Final feature shape for prediction: {final_features.shape}")
    
    # Make prediction
    predictions_scaled = model.predict(final_features, verbose=0)
    print(f"✅ Raw prediction shape: {predictions_scaled.shape}")
    
    # Inverse transform predictions using target scaler
    predictions = target_scaler.inverse_transform(predictions_scaled)
    print(f"✅ Inverse scaled prediction shape: {predictions.shape}")
    
    # Convert to dictionary with target column names
    result = {}
    for i, col_name in enumerate(TARGET_COLS):
        result[col_name] = float(predictions[0][i])
    
    return result

def output_fn(prediction, response_content_type):
    """Format the prediction output"""
    if response_content_type == 'application/json':
        # Return predictions as JSON
        return json.dumps(prediction)
    else:
        raise ValueError(f"Unsupported response content type: {response_content_type}")
'''

# Write inference script to file
inference_script_path = os.path.join(code_dir, "inference.py")
with open(inference_script_path, 'w') as f:
    f.write(inference_script)
print(f"✅ Created inference script: {inference_script_path}")

# 4) Create requirements.txt
# =========================
requirements = '''pillow==9.5.0
'''

requirements_path = os.path.join(code_dir, "requirements.txt")
with open(requirements_path, 'w') as f:
    f.write(requirements)
print(f"✅ Created requirements.txt: {requirements_path}")


✅ Created inference script: bmi_model_work/code/inference.py
✅ Created requirements.txt: bmi_model_work/code/requirements.txt


In [6]:
# =========================
# 4) Create requirements.txt
# =========================
requirements = '''pillow==9.5.0
'''

requirements_path = os.path.join(code_dir, "requirements.txt")
with open(requirements_path, 'w') as f:
    f.write(requirements)
print(f"✅ Created requirements.txt: {requirements_path}")

✅ Created requirements.txt: bmi_model_work/code/requirements.txt


In [7]:
# =========================
# 5) Convert H5 -> SavedModel
# =========================
model = tf.keras.models.load_model(h5_path, compile=False)
print("✅ Loaded Keras model")

# Create SavedModel directory structure
serving_root = os.path.join(workdir, "model")
version_dir = os.path.join(serving_root, "1")
os.makedirs(version_dir, exist_ok=True)

# Save as TensorFlow SavedModel
tf.saved_model.save(model, version_dir)
print(f"✅ Exported SavedModel to: {version_dir}")

# Copy scalers to serving_root (same level as model/1/)
import shutil
for scaler_file in os.listdir(scaler_dir):
    src = os.path.join(scaler_dir, scaler_file)
    dst = os.path.join(serving_root, scaler_file)
    shutil.copy(src, dst)
    print(f"✅ Copied {scaler_file} to model directory")

# Copy code directory to serving_root
code_dest = os.path.join(serving_root, "code")
# Remove destination if it exists to avoid FileExistsError
if os.path.exists(code_dest):
    shutil.rmtree(code_dest)
shutil.copytree(code_dir, code_dest)
print(f"✅ Copied code directory to model directory")


2025-12-18 17:25:46.432519: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-18 17:25:46.481436: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-18 17:25:46.484376: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

✅ Loaded Keras model
INFO:tensorflow:Assets written to: bmi_model_work/model/1/assets


INFO:tensorflow:Assets written to: bmi_model_work/model/1/assets


✅ Exported SavedModel to: bmi_model_work/model/1
✅ Copied scaler_targets.pkl to model directory
✅ Copied scaler_standard_features.pkl to model directory
✅ Copied scaler_robust_features.pkl to model directory
✅ Copied code directory to model directory


In [8]:
# =========================
# 6) Repack as model.tar.gz
# =========================
local_out_tar = os.path.join(workdir, "savedmodel.tar.gz")
with tarfile.open(local_out_tar, "w:gz") as tar:
    # Add model directory with all contents
    tar.add(serving_root, arcname=".")
print(f"✅ Repacked SavedModel tar: {local_out_tar}")


✅ Repacked SavedModel tar: bmi_model_work/savedmodel.tar.gz


In [9]:
# =========================
# 7) Upload to S3
# =========================
uploaded_s3 = upload_file_to_s3(local_out_tar, SAVEDMODEL_TAR_S3_URI)
print(f"✅ Uploaded SavedModel artifact to: {uploaded_s3}")

✅ Uploaded SavedModel artifact to: s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-18-11-02-51-500/output/savedmodel/model.tar.gz


In [10]:
# =========================
# 8) Deploy to SageMaker endpoint
# =========================
session = sagemaker.Session()
region = boto3.Session().region_name

# Create TensorFlow Model
tf_model = TensorFlowModel(
    model_data=uploaded_s3,
    role=ROLE,
    framework_version=FRAMEWORK_VERSION,
    sagemaker_session=session,
)

# Deploy the model
predictor = tf_model.deploy(
    initial_instance_count=1,
    instance_type=INSTANCE_TYPE,
    endpoint_name=ENDPOINT_NAME,
)

print(f"\n✅ Model successfully deployed to endpoint: {ENDPOINT_NAME}")
print(f"   • Region: {region}")
print(f"   • Artifact: {uploaded_s3}")

----------------------------------------------------*

ERROR:sagemaker:Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-endpoint


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 16>:16                                                                            │
│                                                                                                  │
│   13 )                                                                                           │
│   14                                                                                             │
│   15 # Deploy the model                                                                          │
│ ❱ 16 predictor = tf_model.deploy(                                                                │
│   17 │   initial_instance_count=1,                                                               │
│   18 │   instance_type=INSTANCE_TYPE,                                                            │
│   19 │   endpoint_name=ENDPOINT_NAME,                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/tensorflow │
│ /model.py:369 in deploy                                                                          │
│                                                                                                  │
│   366 │   │   │   msg = "The TensorFlow version %s doesn't support EIA." % self.framework_vers   │
│   367 │   │   │   raise AttributeError(msg)                                                      │
│   368 │   │                                                                                      │
│ ❱ 369 │   │   return super(TensorFlowModel, self).deploy(                                        │
│   370 │   │   │   initial_instance_count=initial_instance_count,                                 │
│   371 │   │   │   instance_type=instance_type,                                                   │
│   372 │   │   │   serializer=serializer,                                                         │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/model.py:1 │
│ 814 in deploy                                                                                    │
│                                                                                                  │
│   1811 │   │   │   │   )                                                                         │
│   1812 │   │   │   │   self.sagemaker_session.update_endpoint(self.endpoint_name, endpoint_conf  │
│   1813 │   │   │   else:                                                                         │
│ ❱ 1814 │   │   │   │   self.sagemaker_session.endpoint_from_production_variants(                 │
│   1815 │   │   │   │   │   name=self.endpoint_name,                                              │
│   1816 │   │   │   │   │   production_variants=[production_variant],                             │
│   1817 │   │   │   │   │   tags=tags,                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py │
│ :6250 in endpoint_from_production_variants                                                       │
│                                                                                                  │
│   6247 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   6248 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   6249 │   │                                                                                     │
│ ❱ 6250 │   │   return self.create_endpoint(                

In [11]:
import boto3
sagemaker_client = boto3.client('sagemaker')

# Delete endpoint
sagemaker_client.delete_endpoint(EndpointName='BMI-predcitor-v8')

# Delete endpoint config (optional but recommended)
sagemaker_client.delete_endpoint_config(EndpointConfigName='BMI-predcitor-v8')

{'ResponseMetadata': {'RequestId': 'b9cdb434-59a4-4848-957f-5063b3879404',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b9cdb434-59a4-4848-957f-5063b3879404',
   'strict-transport-security': 'max-age=47304000; includeSubDomains',
   'x-frame-options': 'DENY',
   'content-security-policy': "frame-ancestors 'none'",
   'cache-control': 'no-cache, no-store, must-revalidate',
   'x-content-type-options': 'nosniff',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 18 Dec 2025 17:57:30 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [12]:
import os
import tarfile
import boto3
import json
from urllib.parse import urlparse

import tensorflow as tf
import sagemaker
from sagemaker.tensorflow import TensorFlowModel

# =========================
# Config
# =========================
H5_TAR_S3_URI = "s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-18-11-02-51-500/output/model.tar.gz"
ENDPOINT_NAME = "BMI-predcitor-v8"
INSTANCE_TYPE = "ml.g4dn.xlarge"
FRAMEWORK_VERSION = "2.11.0"
SAVEDMODEL_TAR_S3_URI = "s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-18-11-02-51-500/output/savedmodel/model.tar.gz"

# Scaler URIs
ROBUST_SCALER_URI = "s3://ai-bmi-predictor/scalers/scaler_robust_features.pkl"
STANDARD_SCALER_URI = "s3://ai-bmi-predictor/scalers/scaler_standard_features.pkl"
TARGET_SCALER_URI = "s3://ai-bmi-predictor/scalers/scaler_targets.pkl"

# =========================
# Role inference
# =========================
try:
    ROLE  # type: ignore
except NameError:
    try:
        from sagemaker import get_execution_role
        ROLE = get_execution_role()
    except Exception:
        ROLE = None

if not ROLE:
    raise ValueError("ROLE is None. Set ROLE to your SageMaker execution role ARN.")

# =========================
# Helpers
# =========================
def parse_s3_uri(uri: str):
    # Parse S3 URI into bucket and key components
    p = urlparse(uri)
    if p.scheme != "s3" or not p.netloc or not p.path:
        raise ValueError(f"Invalid S3 URI: {uri}")
    return p.netloc, p.path.lstrip("/")

def safe_extract(tar: tarfile.TarFile, path: str):
    # Safely extract tar file preventing path traversal attacks
    abs_path = os.path.abspath(path)
    for member in tar.getmembers():
        member_path = os.path.abspath(os.path.join(path, member.name))
        if not member_path.startswith(abs_path + os.sep) and member_path != abs_path:
            raise Exception(f"Blocked path traversal attempt: {member.name}")
    tar.extractall(path)

def upload_file_to_s3(local_path: str, s3_uri: str):
    # Upload local file to S3
    bucket, key = parse_s3_uri(s3_uri)
    boto3.client("s3").upload_file(local_path, bucket, key)
    return f"s3://{bucket}/{key}"

def download_from_s3(s3_uri: str, local_path: str):
    # Download file from S3 to local path
    bucket, key = parse_s3_uri(s3_uri)
    boto3.client("s3").download_file(bucket, key, local_path)
    print(f"✅ Downloaded: {s3_uri} to {local_path}")

# =========================
# 1) Download & extract .h5 from tar.gz
# =========================
workdir = "bmi_model_work"
os.makedirs(workdir, exist_ok=True)

local_in_tar = os.path.join(workdir, "model.tar.gz")
extract_dir = os.path.join(workdir, "extracted")
os.makedirs(extract_dir, exist_ok=True)

# Download model tar.gz from S3
in_bucket, in_key = parse_s3_uri(H5_TAR_S3_URI)
boto3.client("s3").download_file(in_bucket, in_key, local_in_tar)
print(f"✅ Downloaded: {H5_TAR_S3_URI}")

# Extract tar.gz file
with tarfile.open(local_in_tar, "r:gz") as tar:
    safe_extract(tar, extract_dir)

# Find the .h5 file in extracted directory
h5_path = None
for root, _, files in os.walk(extract_dir):
    for f in files:
        if f.lower().endswith((".h5", ".hdf5")):
            h5_path = os.path.join(root, f)
            break
    if h5_path:
        break

if not h5_path:
    raise ValueError("❌ No .h5 found inside the downloaded archive.")

print(f"✅ Found H5: {h5_path}")

# =========================
# 2) Download scalers from S3
# =========================
scaler_dir = os.path.join(workdir, "scalers")
os.makedirs(scaler_dir, exist_ok=True)

# Download robust scaler for CNN features
robust_scaler_path = os.path.join(scaler_dir, "scaler_robust_features.pkl")
download_from_s3(ROBUST_SCALER_URI, robust_scaler_path)

# Download standard scaler for height
standard_scaler_path = os.path.join(scaler_dir, "scaler_standard_features.pkl")
download_from_s3(STANDARD_SCALER_URI, standard_scaler_path)

# Download target scaler for inverse transformation
target_scaler_path = os.path.join(scaler_dir, "scaler_targets.pkl")
download_from_s3(TARGET_SCALER_URI, target_scaler_path)

# =========================
# 3) Create inference.py script
# =========================
code_dir = os.path.join(workdir, "code")
os.makedirs(code_dir, exist_ok=True)

inference_script = '''
import os
import json
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import EfficientNetB7, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
from PIL import Image
import io
import base64
import logging

# Set up logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Define target columns in the expected output order
TARGET_COLS = [
    'ankle', 'arm-length', 'bicep', 'calf', 'chest', 'forearm', 'hip',
    'leg-length', 'shoulder-breadth', 'shoulder-to-crotch', 'thigh',
    'waist', 'wrist', 'weight_kg'
]

# Global variables to store loaded models and scalers
feature_extractor = None
bmi_model = None
robust_scaler = None
standard_scaler = None
target_scaler = None

def model_fn(model_dir):
    """Load the model and scalers when the container starts"""
    global feature_extractor, bmi_model, robust_scaler, standard_scaler, target_scaler
    
    try:
        logger.info(f"Loading models and scalers from: {model_dir}")
        logger.info(f"Directory contents: {os.listdir(model_dir)}")
        
        # Load EfficientNetB7 for feature extraction
        logger.info("Loading EfficientNetB7...")
        base_model = EfficientNetB7(weights='imagenet', include_top=True)
        # Create feature extractor using second-to-last layer
        feature_extractor = Model(
            inputs=base_model.inputs,
            outputs=base_model.layers[-2].output
        )
        logger.info("✅ Loaded EfficientNetB7 feature extractor")
        
        # Load the BMI prediction model (TensorFlow SavedModel format)
        model_path = os.path.join(model_dir, 'tensorflow', 'saved_model', '0')
        if not os.path.exists(model_path):
            # Try alternative path structure
            model_path = os.path.join(model_dir, '1')
        if not os.path.exists(model_path):
            # Try another alternative
            model_path = model_dir
            
        logger.info(f"Loading BMI model from: {model_path}")
        bmi_model = tf.keras.models.load_model(model_path, compile=False)
        logger.info(f"✅ Loaded BMI model")
        
        # Load scalers
        logger.info("Loading scalers...")
        
        # Load robust scaler for CNN extracted features
        robust_scaler_path = os.path.join(model_dir, 'scaler_robust_features.pkl')
        with open(robust_scaler_path, 'rb') as f:
            robust_scaler = pickle.load(f)
        logger.info("✅ Loaded robust scaler")
        
        # Load standard scaler for height feature
        standard_scaler_path = os.path.join(model_dir, 'scaler_standard_features.pkl')
        with open(standard_scaler_path, 'rb') as f:
            standard_scaler = pickle.load(f)
        logger.info("✅ Loaded standard scaler")
        
        # Load target scaler for inverse transformation
        target_scaler_path = os.path.join(model_dir, 'scaler_targets.pkl')
        with open(target_scaler_path, 'rb') as f:
            target_scaler = pickle.load(f)
        logger.info("✅ Loaded target scaler")
        
        logger.info("✅ All models and scalers loaded successfully")
        return bmi_model
        
    except Exception as e:
        logger.error(f"Error loading model: {str(e)}")
        raise

def extract_features_from_image(image_data, target_size=(600, 600)):
    """Extract features from a single image using EfficientNetB7"""
    # Convert bytes to PIL Image
    img = Image.open(io.BytesIO(image_data))
    # Convert to RGB if necessary
    if img.mode != 'RGB':
        img = img.convert('RGB')
    # Resize to target size
    img = img.resize(target_size)
    # Convert to array
    img_array = img_to_array(img)
    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)
    # Preprocess for EfficientNet
    img_array = preprocess_input(img_array)
    # Extract features
    features = feature_extractor.predict(img_array, verbose=0)
    # Flatten to 1D array
    return features.flatten()

def input_fn(request_body, request_content_type):
    """Parse input data"""
    try:
        if request_content_type == 'application/json':
            # Parse JSON input
            input_data = json.loads(request_body)
            
            # Decode base64 images
            front_img_data = base64.b64decode(input_data['front_image'])
            side_img_data = base64.b64decode(input_data['side_image'])
            
            # Get gender and height
            gender = input_data['gender']  # expects 'male' or 'female' string
            height_cm = float(input_data['height_cm'])
            
            return {
                'front_image': front_img_data,
                'side_image': side_img_data,
                'gender': gender,
                'height_cm': height_cm
            }
        else:
            raise ValueError(f"Unsupported content type: {request_content_type}")
    except Exception as e:
        logger.error(f"Error in input_fn: {str(e)}")
        raise

def predict_fn(input_data, model):
    """Perform prediction with preprocessing and postprocessing"""
    try:
        logger.info("Starting prediction...")
        
        # Extract features from front mask image
        front_features = extract_features_from_image(input_data['front_image'])
        logger.info(f"✅ Extracted front features: {front_features.shape}")
        
        # Extract features from side mask image
        side_features = extract_features_from_image(input_data['side_image'])
        logger.info(f"✅ Extracted side features: {side_features.shape}")
        
        # Encode gender: male=1, female=0
        gender = input_data['gender'].lower()
        gender_encoded = 1 if gender == 'male' else 0
        
        # Get height
        height_cm = input_data['height_cm']
        
        # Combine all features: front + side + gender + height
        combined_features = np.concatenate([
            front_features,
            side_features,
            [gender_encoded],
            [height_cm]
        ])
        
        # Determine feature indices for scaling
        num_front = len(front_features)
        num_side = len(side_features)
        total_cnn_features = num_front + num_side
        
        # Extract CNN features (front + side)
        cnn_features = combined_features[:total_cnn_features]
        # Gender (no scaling needed)
        gender_feature = combined_features[total_cnn_features:total_cnn_features+1]
        # Height
        height_feature = combined_features[total_cnn_features+1:total_cnn_features+2]
        
        # Apply robust scaler to CNN features
        cnn_features_scaled = robust_scaler.transform(cnn_features.reshape(1, -1)).flatten()
        
        # Apply standard scaler to height
        height_scaled = standard_scaler.transform(height_feature.reshape(1, -1)).flatten()
        
        # Combine scaled features: cnn_features + gender + height
        final_features = np.concatenate([
            cnn_features_scaled,
            gender_feature,
            height_scaled
        ])
        
        # Reshape for model input
        final_features = final_features.reshape(1, -1)
        logger.info(f"✅ Final feature shape for prediction: {final_features.shape}")
        
        # Make prediction
        predictions_scaled = model.predict(final_features, verbose=0)
        logger.info(f"✅ Raw prediction shape: {predictions_scaled.shape}")
        
        # Inverse transform predictions using target scaler
        predictions = target_scaler.inverse_transform(predictions_scaled)
        logger.info(f"✅ Inverse scaled prediction shape: {predictions.shape}")
        
        # Convert to dictionary with target column names
        result = {}
        for i, col_name in enumerate(TARGET_COLS):
            result[col_name] = float(predictions[0][i])
        
        return result
        
    except Exception as e:
        logger.error(f"Error in predict_fn: {str(e)}")
        raise

def output_fn(prediction, response_content_type):
    """Format the prediction output"""
    try:
        if response_content_type == 'application/json':
            # Return predictions as JSON
            return json.dumps(prediction)
        else:
            raise ValueError(f"Unsupported response content type: {response_content_type}")
    except Exception as e:
        logger.error(f"Error in output_fn: {str(e)}")
        raise
'''

# Write inference script to file
inference_script_path = os.path.join(code_dir, "inference.py")
with open(inference_script_path, 'w') as f:
    f.write(inference_script)
print(f"✅ Created inference script: {inference_script_path}")

# =========================
# 4) Create requirements.txt
# =========================
requirements = '''pillow==9.5.0
'''

requirements_path = os.path.join(code_dir, "requirements.txt")
with open(requirements_path, 'w') as f:
    f.write(requirements)
print(f"✅ Created requirements.txt: {requirements_path}")

# =========================
# 5) Convert H5 -> SavedModel
# =========================
model = tf.keras.models.load_model(h5_path, compile=False)
print("✅ Loaded Keras model")

# Create TensorFlow Serving directory structure
serving_root = os.path.join(workdir, "model_package")
tf_serving_dir = os.path.join(serving_root, "tensorflow", "saved_model", "0")
os.makedirs(tf_serving_dir, exist_ok=True)

# Save as TensorFlow SavedModel
tf.saved_model.save(model, tf_serving_dir)
print(f"✅ Exported SavedModel to: {tf_serving_dir}")

# Copy scalers to serving_root (root level for easy access)
import shutil
for scaler_file in os.listdir(scaler_dir):
    src = os.path.join(scaler_dir, scaler_file)
    dst = os.path.join(serving_root, scaler_file)
    shutil.copy(src, dst)
    print(f"✅ Copied {scaler_file} to model package root")

# Copy code directory to serving_root
code_dest = os.path.join(serving_root, "code")
# Remove destination if it exists to avoid FileExistsError
if os.path.exists(code_dest):
    shutil.rmtree(code_dest)
shutil.copytree(code_dir, code_dest)
print(f"✅ Copied code directory to model package")

# =========================
# 6) Repack as model.tar.gz
# =========================
local_out_tar = os.path.join(workdir, "savedmodel.tar.gz")
with tarfile.open(local_out_tar, "w:gz") as tar:
    # Add all contents from serving_root
    for item in os.listdir(serving_root):
        item_path = os.path.join(serving_root, item)
        tar.add(item_path, arcname=item)
print(f"✅ Repacked SavedModel tar: {local_out_tar}")

# Verify tar contents
print("\n📦 Package contents:")
with tarfile.open(local_out_tar, "r:gz") as tar:
    for member in tar.getmembers():
        print(f"  - {member.name}")

# =========================
# 7) Upload to S3
# =========================
uploaded_s3 = upload_file_to_s3(local_out_tar, SAVEDMODEL_TAR_S3_URI)
print(f"\n✅ Uploaded SavedModel artifact to: {uploaded_s3}")

# =========================
# 8) Deploy to SageMaker endpoint
# =========================
session = sagemaker.Session()
region = boto3.Session().region_name

# Create TensorFlow Model with custom inference code
tf_model = TensorFlowModel(
    model_data=uploaded_s3,
    role=ROLE,
    framework_version=FRAMEWORK_VERSION,
    sagemaker_session=session,
)

print(f"\n🚀 Starting deployment to endpoint: {ENDPOINT_NAME}")
print(f"   This may take 5-10 minutes...")

# Deploy the model
predictor = tf_model.deploy(
    initial_instance_count=1,
    instance_type=INSTANCE_TYPE,
    endpoint_name=ENDPOINT_NAME,
)

print(f"\n✅ Model successfully deployed to endpoint: {ENDPOINT_NAME}")
print(f"   • Region: {region}")
print(f"   • Artifact: {uploaded_s3}")


✅ Downloaded: s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-18-11-02-51-500/output/model.tar.gz
✅ Found H5: bmi_model_work/extracted/eff_ann_version8.h5
✅ Downloaded: s3://ai-bmi-predictor/scalers/scaler_robust_features.pkl to bmi_model_work/scalers/scaler_robust_features.pkl
✅ Downloaded: s3://ai-bmi-predictor/scalers/scaler_standard_features.pkl to bmi_model_work/scalers/scaler_standard_features.pkl
✅ Downloaded: s3://ai-bmi-predictor/scalers/scaler_targets.pkl to bmi_model_work/scalers/scaler_targets.pkl
✅ Created inference script: bmi_model_work/code/inference.py
✅ Created requirements.txt: bmi_model_work/code/requirements.txt
✅ Loaded Keras model
INFO:tensorflow:Assets written to: bmi_model_work/model_package/tensorflow/saved_model/0/assets


INFO:tensorflow:Assets written to: bmi_model_work/model_package/tensorflow/saved_model/0/assets


✅ Exported SavedModel to: bmi_model_work/model_package/tensorflow/saved_model/0
✅ Copied scaler_targets.pkl to model package root
✅ Copied scaler_standard_features.pkl to model package root
✅ Copied scaler_robust_features.pkl to model package root
✅ Copied code directory to model package
✅ Repacked SavedModel tar: bmi_model_work/savedmodel.tar.gz

📦 Package contents:
  - scaler_targets.pkl
  - code
  - code/.ipynb_checkpoints
  - code/.ipynb_checkpoints/inference-checkpoint.py
  - code/inference.py
  - code/requirements.txt
  - tensorflow
  - tensorflow/saved_model
  - tensorflow/saved_model/0
  - tensorflow/saved_model/0/assets
  - tensorflow/saved_model/0/fingerprint.pb
  - tensorflow/saved_model/0/saved_model.pb
  - tensorflow/saved_model/0/variables
  - tensorflow/saved_model/0/variables/variables.data-00000-of-00001
  - tensorflow/saved_model/0/variables/variables.index
  - scaler_standard_features.pkl
  - scaler_robust_features.pkl

✅ Uploaded SavedModel artifact to: s3://ai-bmi-

ERROR:sagemaker:Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-endpoint


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 445>:445                                                                          │
│                                                                                                  │
│   442 print(f"   This may take 5-10 minutes...")                                                 │
│   443                                                                                            │
│   444 # Deploy the model                                                                         │
│ ❱ 445 predictor = tf_model.deploy(                                                               │
│   446 │   initial_instance_count=1,                                                              │
│   447 │   instance_type=INSTANCE_TYPE,                                                           │
│   448 │   endpoint_name=ENDPOINT_NAME,                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/tensorflow │
│ /model.py:369 in deploy                                                                          │
│                                                                                                  │
│   366 │   │   │   msg = "The TensorFlow version %s doesn't support EIA." % self.framework_vers   │
│   367 │   │   │   raise AttributeError(msg)                                                      │
│   368 │   │                                                                                      │
│ ❱ 369 │   │   return super(TensorFlowModel, self).deploy(                                        │
│   370 │   │   │   initial_instance_count=initial_instance_count,                                 │
│   371 │   │   │   instance_type=instance_type,                                                   │
│   372 │   │   │   serializer=serializer,                                                         │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/model.py:1 │
│ 814 in deploy                                                                                    │
│                                                                                                  │
│   1811 │   │   │   │   )                                                                         │
│   1812 │   │   │   │   self.sagemaker_session.update_endpoint(self.endpoint_name, endpoint_conf  │
│   1813 │   │   │   else:                                                                         │
│ ❱ 1814 │   │   │   │   self.sagemaker_session.endpoint_from_production_variants(                 │
│   1815 │   │   │   │   │   name=self.endpoint_name,                                              │
│   1816 │   │   │   │   │   production_variants=[production_variant],                             │
│   1817 │   │   │   │   │   tags=tags,                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py │
│ :6250 in endpoint_from_production_variants                                                       │
│                                                                                                  │
│   6247 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   6248 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   6249 │   │                                                                                     │
│ ❱ 6250 │   │   return self.create_endpoint(                